In [1]:
import time
import datetime

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

import pandas as pd

In [11]:
driver = webdriver.Firefox()

In [12]:
driver.get('http://sharesansar.com/price-history')

companyDropdown = driver.find_element_by_id('company')

try:
    elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'company')))
    print("Located")
except TimeoutException:
    print("Too much time")

Located


In [13]:
select = Select(companyDropdown)
options = [x.text for x in select.options]

In [14]:
for option in options:
    print('Currently trying to scrape {}'.format(option))
    driver.find_element_by_id('select2-company-container').click()
    driver.find_element_by_tag_name('body').send_keys(option)
    driver.find_element_by_tag_name('body').send_keys(Keys.ENTER)
    
    driver.find_element_by_id('fromdate').clear()
    driver.find_element_by_id('fromdate').send_keys('2011-01-01')
    
    driver.find_element_by_id('todate').clear()
    driver.find_element_by_id('todate').send_keys('2018-05-07')
    
    Select(driver.find_element_by_id('limit')).select_by_value('500')
    
    driver.find_elements_by_tag_name('button')[2].click()
    
    time.sleep(10)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    table = soup.find_all('table')[0]
    
    if (len(table) > 0):  
        tempDf = pd.read_html(str(table),header=0)[0]
        if (tempDf.shape[0] > 1):
            df = pd.DataFrame(columns=tempDf.columns)
            df = df.append(tempDf).reset_index(drop=True)

            while (tempDf.shape[0] == 500):
                newStart = df.iloc[-1]['Date']
                newStartdt = datetime.datetime.strptime(newStart, '%M/%d/%Y')
                startConv = newStartdt.strftime('%Y-%m-%d')
                driver.find_element_by_id('fromdate').send_keys(newStart)

                time.sleep(10)

                soup = BeautifulSoup(driver.page_source, 'lxml')
                table = soup.find_all('table')[0]

                tempDf = pd.read_html(str(table),header=0)[0]
                df = df.append(tempDf).reset_index(drop=True)
            
            tOption = option.replace('/', '-')
            df.to_csv('data/{}.csv'.format(tOption))
            print('Saved to data/{}.csv'.format(tOption))
            print()

Currently trying to scrape Aakash Capital Limited/ACL
Currently trying to scrape Aankhu Khola Hydropower Company Limited/AANKHU
Currently trying to scrape Aarambha Micro Finance Bittiya Sanstha Limited/AMFI


KeyboardInterrupt: 